# FNN Model Evaluation Notebook

This notebook evaluates a Fully Connected Neural Network (FNN) model for predicting strain responses in layered elastic systems.

## Setup and Dependencies

First, we install required packages and import necessary libraries.

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
import torch
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error,mean_absolute_error
import torch.nn as nn
from torch_geometric.nn import GATConv
import torch.nn.functional as F
import os
import logging
import seaborn as sns

## Data Loading and Preprocessing

This section handles the loading of pre-processed data required for model evaluation:

- **batched_graph_test**: Contains the test dataset in graph format
- **FrameLarge**: Contains the full dataset with material properties and responses
- **Section**: Contains the layered structure information
- **ZS_new**: Contains the z-coordinates (depth points) for each section
- **xs**: Contains the x-coordinates (radial distance points)

The data is loaded from pickle files stored in Google Drive, with paths specified in the configuration. Each dataset contains specific information about the pavement structure and its response to loading.

In [4]:
def load_pickle(path):
    """Load data from pickle file.

    Args:
        path (str): Path to pickle file

    Returns:
        Loaded data object
    """
    with open(path, 'rb') as fp:
        return pickle.load(fp)

batched_graph_test_path = '/content/drive/MyDrive/Graph_Neural_Network/PIML-main/data/batched_graph_test.pkl'
frame_path = '/content/drive/MyDrive/Graph_Neural_Network/PIML-main/data/frame_large.pkl'
section_path = '/content/drive/MyDrive/Graph_Neural_Network/PIML-main/data/section.pkl'
ZS_path = '/content/drive/MyDrive/Graph_Neural_Network/PIML-main/data/ZS.pkl'
xs_path = '/content/drive/MyDrive/Graph_Neural_Network/PIML-main/data/xs.pkl'
model_path = '/content/drive/MyDrive/Graph_Neural_Network/PIML-main/models/GAT_model.pt'


batched_graph_test = load_pickle(batched_graph_test_path)
FrameLarge = load_pickle(frame_path)
Section = load_pickle(section_path)
ZS_new = load_pickle(ZS_path)
xs = load_pickle(xs_path)
checkpoint = torch.load(model_path, map_location='cpu')

<ipython-input-4-d53d7a4d8247>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location='cpu')


## Configuration

This section defines three main configuration dictionaries that control the model and data generation:

### Material Configuration (MATERIAL_CONFIG)
- Defines properties of pavement materials (Asphalt Concrete, Base, Subgrade)
- Specifies ranges for:
  - Thickness (in inches)
  - Modulus (in ksi)
  - Poisson's ratio
  - Number of sublayers
- Sets increment steps for sampling material properties

### Sampling Configuration (SAMPLING_CONFIG)
- Controls data generation parameters:
  - Number of points to generate
  - Points along depth (z-axis) and radial distance (x-axis)
  - Contact radius parameters
  - Train/validation/test split indices
  - Random seed for reproducibility

### Model Configuration (MODEL_CONFIG)
- Defines the GAT model architecture:
  - Input dimension (5 features)
  - Hidden layer dimensions (128, 90)
  - Output dimension (3 strain components)
  - Number of GAT layers (10)

In [5]:
MATERIAL_CONFIG = {
    'N_MATERIALS': 3,
    'MATERIAL_TYPES': ['AC', 'B', 'SG'],  # AC, base, subbase, subgrade
    'SUBLAYER_MAX': [1, 1, 1],  # each material's max sublayers, excluding subgrade
    'THICKNESS_RANGE': [[2, 16], [4, 20]],  # thickness range in inches
    'MODULUS_RANGE': [[500, 2000], [50, 300], [5, 50]],  # modulus range in ksi
    'THICKNESS_INCREMENT': [1, 2, 4],
    'MODULUS_INCREMENT': [50, 20, 20, 5],  # increment in modulus sampling
    'NU_RANGE': [[0.3, 0.4], [0.2, 0.499], [0.2, 0.499]]  # poissons ratio
}

SAMPLING_CONFIG = {
    'N_POINTS': 1000,  # Number of points
    'Z_POINTS': 14,  # points to generate along z
    'X_POINTS': 10,  # points to generate along x
    'A_RANGE': [4, 4],  # contact radius (in)
    'A_POINTS': 1,  # how many contact radii to analyze
    'FACTOR': 0.4,
    'FILTER': 2,
    'SPLIT_IDX': 800,
    'TEST_IDX': 900,
    'SEED': 42
}

MODEL_CONFIG = {
    'INPUT_DIM': 5,
    'HIDDEN_DIM1': 128,
    'HIDDEN_DIM': 90,
    'OUTPUT_DIM': 3,
    'NUM_LAYERS': 10
}

In [6]:
def set_seed(seed):
    """Set seed for reproducibility across all random operations.

    Args:
        seed (int): Random seed value
    """
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed at the start
set_seed(SAMPLING_CONFIG['SEED'])

## Data Generation Functions

This section contains functions for generating and processing the layered elastic system data:

### generatesection()
Generates random pavement sections with realistic material properties:
- Creates multiple layers with different materials
- Assigns random but realistic values for:
  - Layer thicknesses
  - Modulus values
  - Poisson's ratios
- Handles sublayer generation based on thickness constraints
- Returns both section properties and a structured DataFrame

### generate_query_points()
Generates the points where strain will be calculated:
- Creates a grid of points in (x,z) space
- Maps material properties to each point
- Handles boundary conditions between layers
- Returns query points and associated properties

### remove_strain_z()
Filters out sections with unrealistic strain values:
- Removes sections where strain exceeds 1500 µε
- Normalizes z-coordinates
- Returns filtered dataset and updated z-points

In [7]:

def generatesection(N,Nmaterial,MaterialType,Sublayermax,Thicknessrange,Modulusrange,
                              zpoints,xpoints,Thicknessincrement,ModulusIncrement,nurange,arange,apoints):
    """Generate layered elastic system sections with random material properties.

    Args:
        N (int): Number of sections to generate
        Nmaterial (int): Number of material types
        MaterialType (list): List of material type names
        Sublayermax (list): Maximum number of sublayers for each material
        Thicknessrange (list): Thickness ranges for each material
        Modulusrange (list): Modulus ranges for each material
        zpoints (int): Number of points along depth
        xpoints (int): Number of points along radial distance
        Thicknessincrement (list): Increment steps for thickness
        ModulusIncrement (list): Increment steps for modulus
        nurange (list): Poisson's ratio ranges
        arange (list): Contact radius range
        apoints (int): Number of contact radii

    Returns:
        tuple: (Section dictionary, DataFrame with section properties)
    """
    Section={}
    DataFrame={}
    columns=['Structure','Pressure','ContactRadius','z','r']
    thickCol=[f'H{i}' for i in range(1, sum(Sublayermax))]
    columns.extend(thickCol)
    modCol=[f'E{i}' for i in range(1, sum(Sublayermax)+1)]
    columns.extend(modCol)
    nuCol=[f'nu{i}' for i in range(1, sum(Sublayermax)+1)]
    columns.extend(nuCol)

    for sect in range(N):
        Section[sect]={}
        DataFrame[sect]=np.zeros(len(columns))
        #To generate we have to follow some rules as described above
        #First, decide on number of materials
        MatNum=np.random.randint(2,Nmaterial+1)

        #Then on the thickness
        Thick=[]
        Poisson=[]
        Material=[]
        #for each material, we will have poissons ratio and thickness


        for i in range(MatNum-1):
            T=np.arange(Thicknessrange[i][0],Thicknessrange[i][1]+Thicknessincrement[i],Thicknessincrement[i]) #Increments of 0.5in
            P=np.arange(nurange[i][0],nurange[i][1]+0.001,0.05) #Increments of 0.05
            Thick.append(np.random.choice(T))
            Poisson.append(np.random.choice(P))
            Material.append(MaterialType[i])
        #For subgrade
        P=np.arange(nurange[-1][0],nurange[-1][1]+0.001,0.05)
        Poisson.append(np.random.choice(P))
        Material.append(MaterialType[-1])

        #Round the thickness and poissons ratio
        Thick=np.round(Thick,2)
        Poisson=np.round(Poisson,3)

        #Now we can decide on sublayers
        ThickSub=[]
        MaterialSub=[]
        ModulusSub=[]
        PoissonSub=[]
        for i in range(MatNum-1):
            subs=np.random.randint(1,Sublayermax[i]+1)
            M=np.arange(Modulusrange[i][0],Modulusrange[i][1]+ModulusIncrement[i],ModulusIncrement[i]) #increments of 50ksi
            Modulus0=np.random.choice(M)

            if Thick[i]/subs<1: #if smaller than 1 in, no sublayers
                MaterialSub.append(MaterialType[i])
                ThickSub.append(Thick[i])
                ModulusSub.append(Modulus0)
                PoissonSub.append(Poisson[i])

                continue

            for j in range(subs): #else, divide into sublayers
                MaterialSub.append(MaterialType[i])
                ThickSub.append(Thick[i]/subs)
                PoissonSub.append(Poisson[i])

                if j==0: #if we are at the first sublayer assign modulus 0
                    ModulusSub.append(Modulus0)
                else: #else, assign a smaller modulus
                    Modulus0=np.random.uniform(low=Modulusrange[i][0], high=Modulus0)
                    ModulusSub.append(Modulus0)

        #For subgrade
        ModulusSub.append(np.round(np.random.choice(np.arange(Modulusrange[-1][0],Modulusrange[-1][1]+ModulusIncrement[-1],ModulusIncrement[-1]))))
        PoissonSub.append(Poisson[-1])
        MaterialSub.append('SG')
        #Round the values
        ThickSub=np.round(ThickSub,2)
        ModulusSub=np.round(ModulusSub)
        PoissonSub=np.round(PoissonSub,3)

        #To create the dictionary
        Section[sect]['Material']=Material
        Section[sect]['Thickness']=Thick
        Section[sect]['Poisson']=Poisson
        Section[sect]['MaterialSub']=MaterialSub
        Section[sect]['ThicknessSub']=ThickSub
        Section[sect]['PoissonSub']=PoissonSub
        Section[sect]['ModulusSub']=ModulusSub

        #To create the dataframe
        t=np.append(Section[sect]['ThicknessSub'], np.zeros(sum(Sublayermax)-1-len(Section[sect]['ThicknessSub'])))
        m=np.insert(Section[sect]['ModulusSub'], -1, np.zeros(sum(Sublayermax)-len(Section[sect]['ModulusSub'])))
        p=np.insert(Section[sect]['PoissonSub'], -1, np.zeros(sum(Sublayermax)-len(Section[sect]['PoissonSub'])))
        DataFrame[sect]=np.append(np.zeros(5),t)
        DataFrame[sect][0]=sect+1 #assign structure
        DataFrame[sect][1]=80 #assign pressure of 80 psi (9000/np.pi/6**2)
        DataFrame[sect]=np.append(DataFrame[sect],m)
        DataFrame[sect]=np.append(DataFrame[sect],p)
    Frame=pd.DataFrame.from_dict(DataFrame, orient='index',columns=columns)
    return Section,Frame

def generate_query_points(Section, N,xpoints,zpoints,factor,arange,Frame):
  """Generate query points for strain calculation.

    Args:
        Section (dict): Section properties
        N (int): Number of sections
        xpoints (int): Number of x-points
        zpoints (int): Number of z-points
        factor (float): Sampling factor
        arange (list): Contact radius range
        Frame (DataFrame): Input frame with section data

    Returns:
        tuple: Various data structures for query points and properties
  """
  FrameLarge_temp=[]
  final_dict_ztoE=[]
  final_dict_ztoH=[]
  final_dict_ztonu = []
  ZS=[]
  H=[]
  E=[]
  NU = []

  for i in range(N):
    dict_z_to_E = {}
    dict_z_to_H = {}
    dict_z_to_nu={}
    th=sum(Section[i]['Thickness'])+12
    zs=np.power(np.linspace(np.sqrt(0.5),np.power(th,factor),zpoints),1/factor) #sampling near the surface
    zs=np.sort(np.append(zs,np.append(np.cumsum(Section[i]['ThicknessSub'])+0.01,np.cumsum(Section[i]['ThicknessSub'])-0.01)))
    ZS.append(zs)
    E_per_section=np.zeros(len(zs))
    H_per_section = np.zeros(len(zs))
    nu_per_section = np.zeros(len(zs))
    points_above_boundary=np.cumsum(Section[i]['ThicknessSub'])+0.01
    points_below_boundary=np.cumsum(Section[i]['ThicknessSub'])-0.01
    j=0

    while j<=len(Section[i]['ModulusSub'])-1:
      if j==0:
        E_per_section[(zs<=points_below_boundary[j])]=Section[i]['ModulusSub'][j]
        H_per_section[(zs<=points_below_boundary[j])]=Section[i]['ThicknessSub'][j]
        nu_per_section[(zs<=points_below_boundary[j])]=Section[i]['PoissonSub'][j]
        ind=0
        j+=1
      elif j>0 and j<len(Section[i]['ModulusSub'])-1:
        E_per_section[(zs>=points_above_boundary[ind])&(zs<=points_below_boundary[j])]=Section[i]['ModulusSub'][j]
        H_per_section[(zs>=points_above_boundary[ind])&(zs<=points_below_boundary[j])]=Section[i]['ThicknessSub'][j]
        nu_per_section[(zs>=points_above_boundary[ind])&(zs<=points_below_boundary[j])]=Section[i]['PoissonSub'][j]
        ind+=1
        j+=1
      elif j>=len(Section[i]['ModulusSub'])-1:
        E_per_section[(zs>=points_above_boundary[ind])]=Section[i]['ModulusSub'][j]
        H_per_section[(zs>=points_above_boundary[ind])]=-1
        nu_per_section[(zs>=points_above_boundary[ind])]=Section[i]['PoissonSub'][j]
        j+=1

    for z,h in zip(zs,H_per_section):
      dict_z_to_H[z] = h/100

    final_dict_ztoH.append(dict_z_to_H)

    for z,e in zip(zs,E_per_section):
      dict_z_to_E[z]=e/100

    final_dict_ztoE.append(dict_z_to_E)

    for z,nu in zip(zs,nu_per_section):
      dict_z_to_nu[z] = nu

    final_dict_ztonu.append(dict_z_to_nu)

    H.append(H_per_section)
    E.append(E_per_section)
    NU.append(nu_per_section)
    xs = np.linspace(np.sqrt(0.5), np.sqrt(10), xpoints)**2
    #radi=np.sort(random.sample(range(arange[0],arange[1]),apoints))
    radi=[arange[0]]
    Section[i]['z']=zs
    Section[i]['x']=xs
    Section[i]['a']=radi
    FrameTemp=deepcopy(Frame.loc[i:i,:])
    FrameTemp=pd.DataFrame(np.repeat(FrameTemp.values, len(radi)*len(zs)*len(xs), axis=0))
    FrameTemp.columns = Frame.columns
    res = np.matrix([[ii, j, k] for ii in radi
                  for j in zs
                  for k in xs])
    FrameTemp.iloc[:,2:5]=res
    FrameLarge_temp.append(FrameTemp)

  FrameLarge_temp = pd.concat(FrameLarge_temp)
  FrameLarge_temp=FrameLarge_temp.reset_index(drop=True)
  FrameLarge_temp[['Displacement_Z', 'Displacement_H', 'Stress_Z', 'Stress_R', 'Stress_T', 'Stress_RZ', 'Strain_Z', 'Strain_R', 'Strain_T']]=0
  return FrameLarge_temp, ZS, xs, E,NU,final_dict_ztoE,H,final_dict_ztoH,final_dict_ztonu

def remove_strain_z(DF):
    """Remove sections with excessive strain values.

    Args:
        DF (DataFrame): Input dataframe with strain values

    Returns:
        tuple: (Filtered z-points, Filtered dataframe)
    """
    ZS_new=[]
    Length= DF["Structure"].unique()

    for structure in Length:
        struct = int(structure) - 1
        filtered = DF[DF["Structure"] == structure]
        inp = filtered.loc[:, ["Strain_Z"]] * 1e6
        # Check if any Strain_Z value is greater than 2000
        if (inp['Strain_Z'] >=1500).any():
            DF = DF[DF["Structure"] != structure]
            continue
        z_val = filtered['z'].unique()
        ZS_new.append(z_val)

    # normalizing
    ZS_new=[ele/20 for ele in ZS_new]
    return ZS_new,DF


In [8]:
Section_temp, Frame = generatesection(
        SAMPLING_CONFIG['N_POINTS'], MATERIAL_CONFIG['N_MATERIALS'],
        MATERIAL_CONFIG['MATERIAL_TYPES'], MATERIAL_CONFIG['SUBLAYER_MAX'],
        MATERIAL_CONFIG['THICKNESS_RANGE'], MATERIAL_CONFIG['MODULUS_RANGE'],
        SAMPLING_CONFIG['Z_POINTS'], SAMPLING_CONFIG['X_POINTS'],
        MATERIAL_CONFIG['THICKNESS_INCREMENT'], MATERIAL_CONFIG['MODULUS_INCREMENT'],
        MATERIAL_CONFIG['NU_RANGE'], SAMPLING_CONFIG['A_RANGE'],
        SAMPLING_CONFIG['A_POINTS']
)

FrameLarge_temp, ZS, xs, E, NU, final_dict_ztoE, H, final_dict_ztoH, final_dict_ztonu = generate_query_points(
        Section, SAMPLING_CONFIG['N_POINTS'], SAMPLING_CONFIG['X_POINTS'],
        SAMPLING_CONFIG['Z_POINTS'], SAMPLING_CONFIG['FACTOR'],
        SAMPLING_CONFIG['A_RANGE'], Frame
)
ZS, DF = remove_strain_z(FrameLarge)

In [20]:
def convert_units(xs, zs):
    xs_converted = xs * 2.54
    ZS_converted = [i * 2.54 for i in zs]
    xs = np.round(xs_converted, 2)
    ZS_new = [np.round(zs, 2) for zs in ZS_converted]
    return xs, ZS_new

def build_pred_graph(batched_graph_test, final_y_pred):
    pred_graph = {}
    current_index = 0
    for i in range(batched_graph_test.batch_size):
        res_test = len(batched_graph_test[i].y)
        pred_values = final_y_pred[current_index:current_index+res_test]
        pred_graph[i] = pred_values
        current_index += res_test
    return pred_graph

def setup_plotting():
    # get unique number of experiment
    plot_root = "plots"
    if os.path.exists(plot_root):
        avail_nums = os.listdir(plot_root)
        avail_nums = [-1] + [int(d) for d in avail_nums if d.isdigit()]
        exp_num = max(avail_nums) + 1
    else:
        exp_num = 0
    exp_num = str(exp_num)
    print("Logging in plot_dir {}, number {}".format(plot_root, exp_num))


    exp_plot_dir = os.path.join(plot_root, exp_num)

    os.makedirs(exp_plot_dir, exist_ok=True)

    log_path = os.path.join(plot_root, exp_num, "log.txt")
    logging.basicConfig(filename=log_path,
                    filemode='a',
                    format='%(asctime)s | %(message)s',
                    datefmt='%m-%d %H:%M:%S',
                    level=logging.INFO, force=True)
    logging.info("generating plots for experiment {}".format(exp_num))

    return exp_plot_dir

plot_dir = setup_plotting()

Logging in plot_dir plots, number 1


### Data Preprocessing and Splitting

This cell handles the preparation of the dataset for training, validation, and testing:

- **Data Splitting:** The dataset is divided into three subsets:
  - **Training set:** First 135,520 samples (80% of data)
  - **Validation set:** Samples 135,520 to 152,720 (10% of data) 
  - **Test set:** Samples 152,720 to 169,799 (10% of data)

- **Feature Selection:** 
  - **Input features (X):** Columns 3-13 (10 features) containing material properties and geometric parameters
  - **Target variables (Y):** Columns 19-22 (3 features) representing strain components

- **Data Standardization:** 
  - Both input features and target variables are standardized using `StandardScaler`
  - This ensures all features have zero mean and unit variance, improving model training stability and convergence
  - The scaler is fit on training data and applied consistently to validation and test sets

In [26]:
x_train = DF.iloc[:135520, 3:13].values
y_train = DF.iloc[:135520, 19:22].values
x_val = DF.iloc[135520:152720, 3:13].values
y_val = DF.iloc[135520:152720, 19:22].values
x_test = DF.iloc[152720:169799, 3:13].values
y_test = DF.iloc[152720:169799, 19:22].values

# Scale the data
scaler = StandardScaler()
scalery = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)
y_train = scalery.fit_transform(y_train)
y_val = scalery.transform(y_val)
y_test = scalery.transform(y_test)


### FNNModel Class Description

The `FNNModel` class implements a Feed-Forward Neural Network architecture for strain prediction in layered materials:

- **Input Layer:** Accepts a configurable number of input features (typically 10 features from material and geometric properties)
- **Hidden Layers:** Four fully connected layers, each with 64 neurons and ReLU activation functions
  - `fc1`: Maps input to 64-dimensional hidden space
  - `fc2`, `fc3`, `fc4`: Intermediate layers maintaining 64-dimensional representations
  - `fc5`: Final layer producing 3 strain component outputs
- **L1 Regularization:** Implements L1 penalty (Lasso regularization) with configurable lambda parameter (default: 1e-5) to prevent overfitting and promote feature sparsity
- **Output Layer:** Produces 3 outputs corresponding to the three strain components (εz, εr, εθ)
- **Purpose:** Designed for regression tasks, this architecture maps material properties and geometric parameters to strain responses in layered elastic systems

The model uses L1 regularization to enhance generalization and prevent overfitting while maintaining computational efficiency for large-scale material analysis.

In [22]:

class FNNModel(nn.Module):
    def __init__(self, input_size: int, l1_lambda: float = 1e-5):
        super(FNNModel, self).__init__()
        self.l1_lambda = l1_lambda
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 3)
        self.act = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.act(self.fc3(x))
        x = self.act(self.fc4(x))
        x = self.fc5(x)
        return x

    def l1_regularization(self) -> torch.Tensor:
        l1_norm = sum(p.abs().sum() for p in self.parameters())
        return self.l1_lambda * l1_norm

 Load Model Checkpoint

In [23]:
model_path = '/content/drive/MyDrive/Graph_Neural_Network/PIML-main/models/FNN_model.pth'

In [24]:
input_size = x_test.shape[1]
model = FNNModel(input_size)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-24-72495daa6031>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

## Model Evaluation

This section evaluates the trained FNN model's performance:

### Evaluation Process
1. Loads the trained model checkpoint
2. Runs inference on test data
3. Calculates multiple metrics:
   - Mean Squared Error (MSE) for each strain component
   - Mean Absolute Error (MAE) for each strain component
   - Mean Absolute Percentage Error (MAPE) for each strain component
4. Logs results for analysis

### Metrics Explanation
- MSE: Measures average squared difference between predictions and actual values
- MAE: Measures average absolute difference
- MAPE: Measures relative error as a percentage
- Each metric is calculated separately for:
  - Vertical strain (εz)
  - Radial strain (εr)
  - Tangential strain (εθ)

In [28]:

with torch.no_grad():
    test_inputs = torch.tensor(x_test,dtype=torch.float32)

test_outputs = model(test_inputs)
predicted_values = test_outputs.detach().numpy()
y_pred=scalery.inverse_transform(predicted_values)
y_plot=scalery.inverse_transform(y_test)
final_y_plot = y_plot*1e6
final_y_pred = y_pred*1e6

plot_dir = setup_plotting()

mse_z = mean_squared_error( final_y_plot[:, 0], final_y_pred[:, 0])
mse_r = mean_squared_error( final_y_plot[:, 1], final_y_pred[:, 1])
mse_t = mean_squared_error(  final_y_plot[:, 2], final_y_pred[:, 2])
mae_z = mean_absolute_error( final_y_plot[:, 0], final_y_pred[:, 0])
mae_r = mean_absolute_error(final_y_plot[:, 1], final_y_pred[:, 1])
mae_t = mean_absolute_error(final_y_plot[:, 2], final_y_pred[:, 2])
print(f"MSE_Z: {mse_z}, MAE_Z: {mae_z}, MSE_R: {mse_r}, MAE_R: {mae_r}, MSE_T: {mse_t}, MAE_T: {mae_t}")



Logging in plot_dir plots, number 3
MSE_Z: 614.5474897858791, MAE_Z: 6.318042938234566, MSE_R: 2.892334838615746, MAE_R: 0.9372502983024377, MSE_T: 3.312390884021762, MAE_T: 0.9982740003532877


## Visualization

This section contains functions for visualizing model predictions and results:

### plot_actual_vs_predicted_FNN()
Creates scatter plots comparing actual vs predicted values:
- Generates separate plots for each strain component
- Includes perfect prediction line (y=x)
- Uses consistent styling and labeling
- Saves plots to specified directory

### plot_heatmaps()
Generates heatmaps of strain distributions:
- Shows strain distribution in (x,z) space
- Creates separate plots for:
  - Actual strain values
  - Predicted strain values
- Includes:
  - Color bars with strain values
  - Proper axis labels and units
  - Consistent styling and formatting
- Saves high-resolution plots for publication

In [32]:
plot_dir = '/content/drive/MyDrive/Graph_Neural_Network/PIML-main/plots'

In [29]:
def plot_actual_vs_predicted(final_y_plot, final_y_pred, plot_dir=None):
    sns.set_style("darkgrid")
    labels = ['z', 'r', 't']
    for i, label in enumerate(labels):
        plt.scatter(final_y_plot[:, i], final_y_pred[:, i])
        p1 = max(final_y_plot[:, i].max(), final_y_pred[:, i].max())
        p2 = min(final_y_plot[:, i].min(), final_y_pred[:, i].min())
        plt.plot([p2, p1], [p2, p1], color='black', linestyle='--')
        plt.xlabel(f'Actual $\\mu\\epsilon_{label}$', fontsize=12)
        plt.ylabel(f'Predicted $\\mu\\epsilon_{label}$', fontsize=12)
        plt.title(f'Actual vs Predicted in $\\mu\\epsilon_{label}$', fontsize=12)
        plt.tick_params(axis='both', which='major', labelsize=12)
        plt.tight_layout()
        if plot_dir:
            plt.savefig(os.path.join(plot_dir, f"actual_vs_pred_{label}_fnn.png"))
        plt.close()

In [31]:
def plot_heatmaps(ZS_new, xs, batched_graph_test, pred_graph, plot_dir=None, test_struct=0, test_g_struct=0):
    # Strain_Z actual
    sns.set_theme(rc={'figure.figsize':(30,20)}, font_scale=3)
    z = np.array(ZS_new[test_struct])
    response = 'Strain_Z'
    A_prep = np.reshape(batched_graph_test[test_g_struct].y[:,0], (len(ZS_new[test_struct]), len(xs)))
    heatmap = sns.heatmap(A_prep, linewidths=.5, xticklabels=xs, yticklabels=-z, cbar_kws={'label': 'Strain (µε)'})
    colorbar = heatmap.collections[0].colorbar
    colorbar.ax.yaxis.label.set_size(30)
    plt.xlabel('x (cm)', fontsize=30)
    plt.ylabel('z (cm)', fontsize=30)
    plt.title(response)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.tight_layout()
    if plot_dir:
        plt.savefig(os.path.join(plot_dir, "FNN_strain_z_heatmap_struct0_actual.png"))
    plt.close()

    # Strain_Z predicted
    sns.set_theme(rc={'figure.figsize':(20,10)}, font_scale=3)
    A_bar = np.reshape(pred_graph[test_struct][:,0], (len(ZS_new[test_struct]), len(xs)))
    heatmap = sns.heatmap(A_bar, linewidths=.5, xticklabels=xs, yticklabels=-z, cbar_kws={'label': 'Strain (µε)'})
    colorbar = heatmap.collections[0].colorbar
    colorbar.ax.yaxis.label.set_size(30)
    plt.xlabel('x (cm)', fontsize=30)
    plt.ylabel('z (cm)', fontsize=30)
    plt.title('$\\epsilon_z$')
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.tight_layout()
    if plot_dir:
        plt.savefig(os.path.join(plot_dir, "FNN_strain_z_heatmap_struct0_pred.png"))
    plt.close()

    # Strain_R actual
    sns.set_theme(rc={'figure.figsize':(30,20)}, font_scale=3)
    response = 'Strain_R'
    A_prep = np.reshape(batched_graph_test[test_g_struct].y[:,1], (len(ZS_new[test_struct]), len(xs)))
    heatmap = sns.heatmap(A_prep, linewidths=.5, xticklabels=xs, yticklabels=-z, cbar_kws={'label': 'Strain (µε)'})
    colorbar = heatmap.collections[0].colorbar
    colorbar.ax.yaxis.label.set_size(30)
    plt.xlabel('x (cm)', fontsize=30)
    plt.ylabel('z (cm)', fontsize=30)
    plt.title(response)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.tight_layout()
    if plot_dir:
        plt.savefig(os.path.join(plot_dir, "FNN_strain_r_heatmap_struct0_actual.png"))
    plt.close()

    # Strain_R predicted
    sns.set_theme(rc={'figure.figsize':(20,10)}, font_scale=3)
    A_bar = np.reshape(pred_graph[test_struct][:,1], (len(ZS_new[test_struct]), len(xs)))
    heatmap = sns.heatmap(A_bar, linewidths=.5, xticklabels=xs, yticklabels=-z, cbar_kws={'label': 'Strain (µε)'})
    colorbar = heatmap.collections[0].colorbar
    colorbar.ax.yaxis.label.set_size(30)
    plt.xlabel('x (cm)', fontsize=30)
    plt.ylabel('z (cm)', fontsize=30)
    plt.title('$\\epsilon_r$')
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.tight_layout()
    if plot_dir:
        plt.savefig(os.path.join(plot_dir, "FNN_strain_r_heatmap_struct0_pred.png"))
    plt.close()

In [33]:
# Convert units
xs, ZS_new = convert_units(xs, ZS_new)

# Build prediction graph
pred_graph = build_pred_graph(batched_graph_test, predicted_values)

# Plot heatmaps
plot_heatmaps(ZS_new, xs, batched_graph_test, pred_graph, plot_dir=plot_dir)

In [35]:
plot_actual_vs_predicted(final_y_plot, final_y_pred, plot_dir=plot_dir)
